In [1]:
import os
from langchain.agents import tool
import requests
import json
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent, ToolNode
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
load_dotenv()

@tool
def get_weather(city: str) -> str:
  """Get the weather for a given city."""
  return f"It's always sunny in {city}!"

@tool
def get_smiles_exchange_rates(from_currency: str, to_currency: str):
    """Get exchange rate for a given currency pair."""
    SMILES_EXCHANGE_JP = os.getenv("SMILES_EXCHANGE_JP")
    if not SMILES_EXCHANGE_JP:
        raise ValueError("SMILES_EXCHANGE_JP environment variable is not set.")
    response = requests.get(SMILES_EXCHANGE_JP)
    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code}")
    response_json = response.json()
    rates = json.loads(response_json["Rates"])
    currency = rates["ALL_ALL_ALL"]["Currency"]
    print(currency)

    exchange_rate_data = []
    for value in currency:
        data = currency[value]
        exchange_rate_data.append(
            {
                "sender_unit": float(1),
                "sender_currency": data["From"],
                "receiver_unit": float(data["SellingRate"]),
                "receiver_currency": data["To"],
            }
        )
    for rate in exchange_rate_data:
      if rate["sender_currency"] == from_currency and rate["receiver_currency"] == to_currency:
        return f"%s" % rate["receiver_unit"]
    return f"Sorry we don't have exchange rate for {from_currency} to {to_currency}."

## Define LLM, agent_node & tool_node
llm = ChatOllama(model="llama3.2", temperature=0.3)
agent_node = create_react_agent(llm, [get_weather, get_smiles_exchange_rates])
tool_node = ToolNode([get_weather, get_smiles_exchange_rates])

## Init StateGraph, and add nodes (agent, tool) to the graph
graph = StateGraph(MessagesState)
graph.add_node("agent", agent_node)
graph.add_node("tools", tool_node)

## Connect START EDGE to Agent Node, describing the flow of application
graph.add_edge(START, "agent")

def agent_router(state):
  """
  Agent router loops checks if tool calling is required, loops through them with values, and returns
  """
  messages = state["messages"]
  last = messages[-1]
  if getattr(last, "tool_call", None) or getattr(last, "tool_calls", None):
      return "tools"
  return END

## Hence, these edges would be conditional, based on the agent_router logic and user prompt
graph.add_conditional_edges("agent", path=agent_router)

## Finally, connect tools node to agent node
graph.add_edge("tools", "agent")
app = graph.compile(checkpointer=MemorySaver())

msg = [HumanMessage(content="What's exchange rate of Japan to Nepal? Alos, what's the weather like in Kathmandu?")]
config: RunnableConfig = {"configurable": {"thread_id": "thread_1"}}

response = app.invoke({"messages": msg}, config=config)
print(response["messages"][-1])

{'Currency_USD_JPY': {'From': 'USD', 'To': 'JPY', 'BuyingRate': None, 'SellingRate': 141.48, 'BookRate': 143.64, 'AverageRate': None, 'TTMRate': None}, 'Currency_JPY_USD': {'From': 'JPY', 'To': 'USD', 'BuyingRate': None, 'SellingRate': 0.006914, 'BookRate': 0.006962, 'AverageRate': None, 'TTMRate': None}, 'Currency_JPY_PHP': {'From': 'JPY', 'To': 'PHP', 'BuyingRate': None, 'SellingRate': 0.387, 'BookRate': 0.3875, 'AverageRate': None, 'TTMRate': None}, 'Currency_PHP_JPY': {'From': 'PHP', 'To': 'JPY', 'BuyingRate': None, 'SellingRate': 2.5419, 'BookRate': 2.5806, 'AverageRate': None, 'TTMRate': None}, 'Currency_JPY_IDR': {'From': 'JPY', 'To': 'IDR', 'BuyingRate': None, 'SellingRate': 113.7, 'BookRate': 113.7853, 'AverageRate': None, 'TTMRate': None}, 'Currency_IDR_JPY': {'From': 'IDR', 'To': 'JPY', 'BuyingRate': None, 'SellingRate': 0.008657, 'BookRate': 0.008788, 'AverageRate': None, 'TTMRate': None}, 'Currency_JPY_VND': {'From': 'JPY', 'To': 'VND', 'BuyingRate': None, 'SellingRate': 1